In [2]:
from yahoofinancials import YahooFinancials
import numpy as np
import pandas as pd
import pickle

In [3]:
nifty_200 = pd.read_csv('database\ind_nifty200list.csv')

In [1]:
start_date = '2014-01-01'
end_date = '2020-09-30'

price_df = pd.DataFrame()
for stock in nifty_200.Symbol:
    print("fetching data for "+stock)
    try:
        current_price_data = pd.DataFrame(YahooFinancials(stock+'.NS').get_historical_price_data(start_date, end_date, 'monthly')[stock+'.NS']['prices'])
        current_price_data['MarketCap'] = current_price_data['adjclose'] * YahooFinancials(stock+'.NS').get_num_shares_outstanding()
        current_price_data['Symbol'] = stock
        price_df = price_df.append(current_price_data)
    except:
        print("data not available for "+stock)

In [5]:
price_df.to_pickle('database\\nifty_200_stocks_price_data_24Oct20.pickle')

In [4]:
price_df = pd.read_pickle('database\\nifty_200_stocks_price_data_24Oct20.pickle')

In [10]:
master_df = price_df.copy()
master_df['GrossProfit'] = 0
master_df['BookValue'] = 0
for stock in nifty_200.Symbol:
    print("fetching data for "+stock)
    try:
        inc_stmt = YahooFinancials(stock+'.NS').get_financial_stmts('annual', 'income')['incomeStatementHistory'][stock+'.NS']
        balance_sheet = YahooFinancials(stock+'.NS').get_financial_stmts('annual', 'balance')['balanceSheetHistory'][stock+'.NS']
        for i in range(len(inc_stmt)):
            current_date = list(inc_stmt[i].keys())[0]
            gross_profit = inc_stmt[i][current_date]['grossProfit']
            book_value = balance_sheet[i][current_date]['commonStock']
            if i > 0:
                previous_date = list(inc_stmt[i-1].keys())[0]
                master_df.loc[(master_df.formatted_date>=current_date) & (master_df['Symbol'] == stock) & (master_df['formatted_date'] < previous_date),['GrossProfit']] = gross_profit
                master_df.loc[(master_df.formatted_date>=current_date) & (master_df['Symbol'] == stock)  & (master_df['formatted_date'] < previous_date) ,['BookValue']] = book_value
            else:
                master_df.loc[(master_df.formatted_date>=current_date) & (master_df['Symbol'] == stock),['GrossProfit']] = gross_profit
                master_df.loc[(master_df.formatted_date>=current_date) & (master_df['Symbol'] == stock) ,['BookValue']] = book_value
    except:
        print("data not available for "+stock)

In [8]:
master_df.to_pickle('database\master_df_data_24Oct20.pickle')